# 清洗資料，進行分析

In [3]:
import os
file_path = 'my_log'
file_log = 'MR1'
file_log_new = 'MR_new'


# include_line=('Newtonsoft.Json' or
#                     'String ' or
#                     'Rev:{"code"' or
#                     'Send:https' or
#                     'System.N')

exclude_keyword=[
    'Newtonsoft.Json', 
    'String ',
    'Rev:{"code',
    'Send:https',
    'System.N',
    '於',
    '券商回覆',
    '下單數量',
    '未定義',
    '內部例外',
    '多空比例',
    '獨立口數',
    '非同步下單',
    '同步下單',
    'Login Error',
    'ServerExit'
]

include_keyword = ['B 成交價格','S 成交價格']

f = open(file_path + f'/{file_log}.log',"r") 
lines = f.readlines() 
f.close()

f = open(file_path + f'/{file_log_new}.log',"w") 
for line in lines:
    match_exclude_keyword = False
    for word in exclude_keyword:
            if word in line:
                match_exclude_keyword = True
    
    match_include_keyword = False
    for word in include_keyword:
        if word in line:
            match_include_keyword = True
                
    if match_exclude_keyword == False and match_include_keyword == True:
        line = line.replace('[DEBUG] ',',')
        line = line.replace('[\x08.\x02]','')
        line = line.replace(' 成交價格：',' , ')
        line = line.replace(' 成交數量：',' , ')
        line = line.replace(' , ',' *** ')
        line = line.replace('[','')
        line = line.replace(']','')
        f.write(line) 
f.close()
print('整理完畢')

整理完畢
